# Creating a Chatbot linked to the Database

## 1. Importing libraries

In [19]:
%pip install python-dotenv==1.0.1 surrealdb==1.0.3 openai==1.63.0

Note: you may need to restart the kernel to use updated packages.


In [20]:
from surrealdb import Surreal
from dotenv import load_dotenv
from openai import OpenAI
import os
import json

load_dotenv(override=True)

True

## 2. Connect to the database

In [21]:
db = Surreal(os.getenv("SURREAL_URL"))
db.signin({"username": os.getenv("SURREAL_USER"), "password": os.getenv("SURREAL_PASS")})
db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))

print(db.version())

surrealdb-2.2.0


## 4. Embed articles and save to database

In [23]:
client = OpenAI() # api key loaded from .env
embed_model_name = "text-embedding-3-small"
llm_model_name = "gpt-4o"
system_prompt = """
You are an Algerian legal expert AI trained to provide accurate answers.
You must strictly base your answer on the provided context.
You must avoid at all cost to mention in your answer that you are being provided with context.
You must strictly answer in Arabic.
Translate the keywords from context to Arabic.
Your response must adhere to any requested format in the question, if present.
Interpret the context to extract required information and present it directly, without mentioning the reasoning process.
If not, state clearly that you are unable to answer the question.
Do not fabricate or assume facts.
""".strip()

def search_articles(query):
    # more about this on https://platform.openai.com/docs/guides/embeddings#how-to-get-embeddings
    response = client.embeddings.create(
            input=query,
            model=embed_model_name,
        )
    q_vector = response.data[0].embedding

    results = db.query("""
        SELECT
            text,
            vector::similarity::cosine(embed_vector, $q_vector) AS score
        FROM articles
        ORDER BY score DESC
        LIMIT 5
        """,
        {"q_vector": q_vector}
    )

    print(json.dumps(results, indent=2, ensure_ascii=False), end="\n\n")
    return results

def ask_question(query):
    articles = search_articles(query)

    user_prompt = "```context\n"
    for article in articles:
        user_prompt += f"{article['text']}\n"
    user_prompt += "```\n"
    user_prompt += f"```question\n{query}\n```"

    print(user_prompt)
    print("\n------------------\n")

    # more about this on https://platform.openai.com/docs/guides/text-generation#quickstart
    stream = client.chat.completions.create(
        model=llm_model_name,
        messages=[
            {"role": "developer", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )
        
    for chunk in stream:
        text_chunk = chunk.choices[0].delta.content
        if text_chunk is None:
            continue
        print(text_chunk, end="")

# question = "if I'm christian, what should I do in Algeria? and give me brief answer"
# question = "Anything about family law"
question = "معلومات عن الانتخابات"
ask_question(question)

[
  {
    "score": 0.4487533001776305,
    "text": "المادة 193 : تُلزم السلطات العمومية المكلفة بتنظيم الانتخابات بإحاطاتها بالشفافية والحياد. وبهذه الصفة، توضع القائمة الانتخابية عند كل انتخاب تحت تصرف المترشحين. يحدد القانون العضوي المتعلق بنظام الانتخابات كيفيات تطبيق هذا الحكم."
  },
  {
    "score": 0.41368931099193795,
    "text": "المادة 184 : يجب على أعضاء المجلس الدستوري المنتخبين أو المعينين ما يأتي : - بلوغ سن أربعين (04) سنة كاملة يوم تعيينهم أو انتخابهم، - التمتع بخبرة مهنية مدتها خمس عشرة (51) سنة على الأقل في التعليم العالي في العلوم القانونية، أو في القضاء،  أو في مهنة محام لدى المحكمة العليا أو لدى مجلس الدولة أو في وظيفة عليا في الدولة."
  },
  {
    "score": 0.40068075642472994,
    "text": "المادة 194 : تُحدث هيئة عليا مستقلة لمراقبة الانتخابات. ترأس الهيئة شخصية وطنية يعينها رئيس الجمهورية، بعد استشارة الأحزاب السياسية. للهيئة العليا لجنة دائمة. وتنشر الهيئة العليا أعضاءها الآخرين فور استدعاء الهيئة الانتخابية. تتكون الهيئة العليا بشكل متساو من : - قضاة يقترحهم الم

### Useful links

- https://platform.openai.com/docs/guides/rate-limits?tier=tier-one